In [ ]:
# different sized culster for different algos for different datasets.

## step1 data subsetting
# 3 domains: collab, ECE, Shanti pages

## port 443 


## step2 
# 3 combination of algos

## step3
# 

In [ ]:
# use random sample as a baseline?
# the comparison needs to be a correct comparison: compare same time period for two different weekdays.
#     have to be the same comparison.
# by hour of the same day? all we know if that the user behaviors are changing.. thus not the same comparison.
# adjust the sample size. to run KSE.




In [1]:
## Tianye Song, Julina Zhang, Kerry Jones, Capstone Cyber Intrusion Detection, Formalized Code
## 3/27/17

## This notebook contains modularized code for our two-step approach
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans

In [159]:
#### Data Preprocessing ####
# read netFlow data
# data1 = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')
# data_collab = data1[(((data1.srcAddr == '128.143.2.75') & (data1.srcPort == 443)) | ((data1.dstAddr =='128.143.2.75') & (data1.dstPort == 443)))]

#### First step is to get anomaly scores from LOF/IForest, append to dataset.
# input a dataframe, an anomaly detection method.
# LOF, IForest, or nothing.
def labeling(df, method):
    # first step is to generate a list of anomaly scores using chosen method.
    temp = df[["bytes",'pkts','dur','rate']]
    if(method == "IForest"):
        clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
        clf.fit(temp)
        y_pred = clf.predict(temp)
        df['ISO_label'] = y_pred
    elif(method == "LOF"):
        clf = LocalOutlierFactor(n_neighbors=20)
        y_pred = clf.fit_predict(temp)
        df['LOF_label'] = y_pred
    return(df)
# take 50% sample data, then label it. 
# data_collab_labeled = labeling(data_collab.sample(frac = 0.5, replace = False), "LOF") 


#### Second step is to apply K-means clustering to the same dataset 
# kmeans = KMeans(n_clusters=553, random_state=0).fit(sample) # select K based on sqrt(n/2)
# result = kmeans.labels_
# data_collab_labeled['cluster_label'] = result
# save the result..
# data_collab_labeled.to_csv("~/Desktop/collab_clean_sample.csv")

# Read in sample data generated by extracint collab-related netFlow traffic.
Collab_sample = pd.read_csv('~/Desktop/collab_clean_sample.csv')[["bytes",'pkts','dur','rate','LOF_label','cluster_label']]
Collab_sample = labeling(Collab_sample, 'IForest')
Collab_sample.head(5)

,bytes,pkts,dur,rate,LOF_label,cluster_label,ISO_label
0,1936,14,0.074,209297.63,1,299,1
1,1500,14,0.065,184615.90,1,438,1
2,3972,16,0.270,117688.89,1,73,1
3,2294,14,0.169,108591.72,1,73,1
4,13030,24,0.100,1042400.99,1,78,1


In [144]:
len(Collab_sample[Collab_sample.LOF_label == -1]) / len(Collab_sample)

0.10000130492511361

In [145]:
len(Collab_sample[Collab_sample.ISO_label == -1]) / len(Collab_sample)
# Number is around 0.05 because we set contamination to be 0.05?

0.05000799266632086

In [160]:
Collab_sample.head()

,bytes,pkts,dur,rate,LOF_label,cluster_label,ISO_label
0,1936,14,0.074,209297.63,1,299,1
1,1500,14,0.065,184615.90,1,438,1
2,3972,16,0.270,117688.89,1,73,1
3,2294,14,0.169,108591.72,1,73,1
4,13030,24,0.100,1042400.99,1,78,1


In [161]:
#### Third step is to make use of the result of step 1 & 2, to reduce misclassified normal instances(reduce false alarm).
# threshold is the maximum percentage of anomalies allowed in df

def normal_profiling(data, method,threshold):
    df = data
    clusterID = df.cluster_label.values
    if(method == 'IForest'):
        df['normalized_ISO_labels'] = df['ISO_label']
        labels = df["cluster_label"].value_counts().index.tolist()
        # when the proportion of anomalies is larger than a threshold
        while (len(df[df.normalized_ISO_labels == -1]) / len(df) >= threshold):
            # set anything that has label x, yet is classified by LOF to be an anomaly to be equal to 1. 
            x = labels.pop(0)
            df.loc[((df['cluster_label'] == x) & (df['normalized_ISO_labels'] == -1)),'normalized_ISO_labels'] = 1

    elif(method == 'LOF'):
        df['normalized_LOF_labels'] = df['LOF_label']
        labels = df["cluster_label"].value_counts().index.tolist()
        # when the proportion of anomalies is larger than a threshold
        while (len(df[df.normalized_LOF_labels == -1]) / len(df) >= threshold):
            # set anything that has label x, yet is classified by LOF to be an anomaly to be equal to 1. 
            x = labels.pop(0)
            df.loc[((df['cluster_label'] == x) & (df['normalized_LOF_labels'] == -1)),'normalized_LOF_labels'] = 1

    return(df)
# case2: apply K-means repeatedly <-- not realistic because time-complexity
# Collab_norm = normal_profiling(Collab_sample, 0.05)
# len(Collab_norm[Collab_norm.normal_profile_labels == -1]) / len(Collab_norm)

In [167]:
Collab_norm_ISO = normal_profiling(Collab_sample,'IForest',0.05)
Collab_norm_LOF = normal_profiling(Collab_sample,'LOF',0.05)

In [168]:
Collab_sample.head()

,bytes,pkts,dur,rate,LOF_label,cluster_label,ISO_label,normalized_ISO_labels,normalized_LOF_labels
0,1936,14,0.074,209297.63,1,299,1,1,1
1,1500,14,0.065,184615.90,1,438,1,1,1
2,3972,16,0.270,117688.89,1,73,1,1,1
3,2294,14,0.169,108591.72,1,73,1,1,1
4,13030,24,0.100,1042400.99,1,78,1,1,1


In [169]:
len(Collab_sample[Collab_sample.normalized_ISO_labels == -1])

21921

In [170]:
len(Collab_sample[Collab_sample.normalized_LOF_labels == -1])

29569

In [177]:
#### Forth step is to use majority vote to get final results.
def maj_vote(df, method):
    anom_list = []
    if(method == "IForest"):
        pool = set(df[df.normalized_ISO_labels == -1].cluster_label) # a set of IDs of all clusters that contain anomalies
        for i in pool:
                # for each of cluster i, calculate proportion of anomaly
                proportion_anomaly = len(df[(df['cluster_label'] == i) & (df['normalized_ISO_labels'] == -1)]) / len(df[df.cluster_label == i])
                if proportion_anomaly > 0.5:
                    anom_list.extend([i])
    elif(method == "LOF"):
        pool = set(df[df.normalized_LOF_labels == -1].cluster_label) # a set of IDs of all clusters that contain anomalies
        for i in pool:
                # for each of cluster i, calculate proportion of anomaly
                proportion_anomaly = len(df[(df['cluster_label'] == i) & (df['normalized_LOF_labels'] == -1)]) / len(df[df.cluster_label == i])
                if proportion_anomaly > 0.5:
                    anom_list.extend([i])
    # do majority vote, and return a list of anomalous clusters
    return(anom_list)

In [178]:
anomaly_clusters_ISO = maj_vote(Collab_sample,'IForest')
anomaly_clusters_LOF = maj_vote(Collab_sample, 'LOF')

In [182]:
print("# of Anom_cluster_ISO: %s , # of Anom_cluster_LOF: %s" % (len(anomaly_clusters_ISO), len(anomaly_clusters_LOF)))

# of Anom_cluster_ISO: 493 , # of Anom_cluster_LOF: 102


In [185]:
len(Collab_sample)

613062

In [187]:
#### Fifth step is to retrieve the anomaly clusters, in the form of a df
frames_ISO = []
for i in anomaly_clusters_ISO:
    cluster = Collab_sample[Collab_sample.cluster_label == i]
    frames_ISO.append(cluster)
anomalies_ISO = pd.concat(frames_ISO)

In [188]:
frames_LOF = []
for i in anomaly_clusters_LOF:
    cluster = Collab_sample[Collab_sample.cluster_label == i]
    frames_LOF.append(cluster)
anomalies_LOF = pd.concat(frames_LOF)

In [190]:
print("# of Anom_ISO: %s , # of Anom_LOF: %s" % (len(anomalies_ISO), len(anomalies_LOF)))

# of Anom_ISO: 15839 , # of Anom_LOF: 347


In [193]:
## only 347 anomalies for LOF..?
len(anomalies_LOF) / len(Collab_sample)

0.0005660112680283561

In [198]:
# for i in frames_LOF:
#     print(len(i))

In [200]:
# for i in frames_ISO:
#     print(len(i))

In [237]:
## seems like there is no problem with the algorithm. 

## So what is going on with those 347 anomalies? They must be really abnormal
min(anomalies_LOF['rate'])

41707.800000000003

In [206]:
anomalies_LOF['MB'] = anomalies_LOF.bytes / 1000000

In [239]:
anomalies_LOF.MB.sort(ascending = False, inplace = False)[0:10]

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  if __name__ == '__main__':


185093    300.512086
543055    297.338748
266898    294.343562
107015    293.306604
520488    250.891910
3861      212.533822
467498    184.196298
99754     171.262766
199796    133.393712
175032    130.847544
Name: MB, dtype: float64

In [214]:
anomalies_ISO['MB'] = anomalies_ISO.bytes / 1000000

In [339]:
Collab_sample.bytes.sort(ascending = False, inplace = False)[1:10]

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  if __name__ == '__main__':


543055    297338748
266898    294343562
107015    293306604
520488    250891910
3861      212533822
467498    184196298
99754     171262766
199796    133393712
175032    130847544
Name: bytes, dtype: int64

In [215]:
anomalies_ISO.MB.sort(ascending = False, inplace = False)[1:10]

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  if __name__ == '__main__':


543055    297.338748
266898    294.343562
107015    293.306604
520488    250.891910
3861      212.533822
467498    184.196298
99754     171.262766
199796    133.393712
175032    130.847544
Name: MB, dtype: float64

In [223]:
# is LOF's result a subset of ISO?
len(set(anomalies_LOF.index).intersection(set(anomalies_ISO.index)))

347

In [273]:
# it appears that LOF's result is a subset of ISO.
# What about naive K-means?
naive = Collab_sample[['bytes','pkts','dur','rate','cluster_label']]
temp = naive['cluster_label'].value_counts(ascending = True)
temp.iloc[1]


1

In [319]:
sum = 0
index = 0
result = 0
while True:
    sum += temp.iloc[index]
    index += 1
    if (sum >= 30653):
        result = temp.iloc[index]
        break
index

510

In [326]:
anomaly_clusters_naive = list(temp.nsmallest(509).index)

In [332]:
len(anomaly_clusters_naive)

509

In [318]:
# let's say the top 5% is anomalies (a naive assumption to be discussed) 
len(Collab_sample) * 0.05

30653.100000000002

In [295]:
## so we want the first 3065 observations 

In [327]:
frames_naive = []
for i in anomaly_clusters_naive:
    cluster = Collab_sample[Collab_sample.cluster_label == i]
    frames_naive.append(cluster)
anomalies_naive = pd.concat(frames_naive)

In [328]:
len(anomalies_naive)

30373

In [329]:
anomalies_naive.head()

,bytes,pkts,dur,rate,LOF_label,cluster_label,ISO_label,normalized_ISO_labels,normalized_LOF_labels
269179,5147590,3646,2.666,15446632.30,-1,235,-1,-1,-1
266898,294343562,207362,148.632,15842809.73,-1,98,-1,-1,-1
543055,297338748,209598,316.131,7524443.93,-1,99,-1,-1,-1
300544,61902764,44030,323.560,1530541.82,-1,102,-1,-1,-1
302500,36964824,26048,18.826,15707988.53,-1,103,-1,-1,-1


In [333]:
len(set(anomalies_naive.index).intersection(set(anomalies_ISO.index)))
# in this case, all of anomalies identified by the Naive K-means approach is a subset of anomalies_ISO

15839

In [336]:
len(anomalies_ISO.index)

15839

In [313]:
len(set(Collab_sample[Collab_sample.normalized_ISO_labels == -1].cluster_label))

552

In [314]:
len(set(Collab_sample[Collab_sample.normalized_LOF_labels == -1].cluster_label))

306

In [316]:
len(Collab_sample[Collab_sample.normalized_ISO_labels == -1]) / len(Collab_sample)

0.035756579269307184

In [317]:
len(Collab_sample[Collab_sample.normalized_LOF_labels == -1]) / len(Collab_sample)

0.04823166335541919

In [32]:
rng = np.random.RandomState(42)
X = 0.3 * rng.randn(100, 2)
X_train = np.r_[X + 2, X - 2]
len(X_train)

200

In [38]:
X = 0.3 * rng.randn(20, 2)
X_test = np.r_[X + 2, X - 2]
len(X_test)

40

In [39]:
X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))
len(X_outliers)

20

In [40]:
# fit the model
clf = IsolationForest(max_samples=100, random_state=rng)
clf.fit(X_train)


IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
        max_samples=100, n_estimators=100, n_jobs=1,
        random_state=<mtrand.RandomState object at 0x112979870>, verbose=0)

In [48]:
y_pred_train = clf.predict(X_train)
sum(y_pred_train == 1)/200

0.90000000000000002

In [49]:
y_pred_test = clf.predict(X_test)
sum(y_pred_test == 1)/40

0.69999999999999996

In [53]:
y_pred_outliers = clf.predict(X_outliers)
sum(y_pred_outliers == 1)/20

0.050000000000000003